In [1]:
%load_ext autoreload
%autoreload 2

from dandi.consts import known_instances
from dandi.dandiapi import DandiAPIClient
from dandi.metadata.nwb import get_metadata
from dandi.organize import CopyMode, FileOperationMode, OrganizeInvalid
from dandi.pynwb_utils import nwb_has_external_links
from dandi.validate_types import Severity
import fsspec
from fsspec.implementations.cached import CachingFileSystem
import pynwb
import h5py

dandiset_id, dandi_path, dandi_instance = ('215528',
    'sub-Winnie/sub-Winnie_ses-Winnie-20220719_obj-xnyc3s_ecephys.nwb',
    'dandi-staging'
)

# get the s3 url from Dandi
with DandiAPIClient(
    dandi_instance=known_instances[dandi_instance],
) as client:
    asset = client.get_dandiset(dandiset_id).get_asset_by_path(
        dandi_path
    )
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

# stream the file from s3
# first, create a virtual filesystem based on the http protocol
fs = fsspec.filesystem("http")

# create a cache to save downloaded data to disk (optional)
fsspec_file = CachingFileSystem(
    fs=fs,
    cache_storage=f".",  # Local folder for cache
)

# Open and return the file
fs_file = fsspec_file.open(s3_url, "rb")
io = pynwb.NWBHDF5IO(file=h5py.File(fs_file))
nwb = io.read()


# select the object we want to display
nwb_obj = nwb.scratch['filtered data']
st = nwb_obj.timestamps[0]
interval = [st,st+300]

import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


In [2]:
from viewer.time_series import NwbTimeSeriesViewer
import numpy as np

channel_groups = {}
for i in range(6):
    channel_groups[i] = np.arange(4)+i*4
# channel_groups = None
view = NwbTimeSeriesViewer(nwb_obj, interval, n_plot=3000, max_channels=20,
                 channel_index=None, image_shape = (512, 512),
                 channel_options = np.arange(25),
                 channel_groups=channel_groups,)
view.compile()
view.run()

2025-04-07 15:43:02.412 python[83146:9171315] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-07 15:43:02.412 python[83146:9171315] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:652: UserWarning: Previous color_by key 'group' not present in features. Falling back to track_id
  warn(
/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:652: UserWarning: Previous color_by key 'group' not present in features. Falling back to track_id
  warn(
/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:652: UserWarning: Previous color_by key 'group' not present in features. Falling back to track_id
  warn(
/Users/samuelbray/miniforge3/envs/spyglass/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:652: UserWarning: Previous color_by key 'group' not present in features. Falli